# <font color='red'>Question 1</font>

# <font color='green'>Working with AIF360 on IBM Watson
    
    - Bias and Fairness analysis on loan defaulters prediction

In [1]:
#Firstly install Aif360 package -> Uncomment the below line
# !pip install aif360
# !pip install cvxpy==0.4.11

In [2]:
# Load all necessary packages

import sys
sys.path.insert(1, "../")  #this will append a new location to accept environment variables from

import os

import numpy as np
np.random.seed(0)

from aif360.datasets import GermanDataset #Data set

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

from IPython.display import Markdown, display

In [3]:
#Adding a location to read data from

aif360_location = !python -c "from distutils.sysconfig import get_python_lib; print(get_python_lib())"

install_loc = os.path.join(aif360_location[0], "aif360/data/raw/german/")

#Checking the current directory
%cd $install_loc

/opt/conda/envs/Python36/lib/python3.6/site-packages/aif360/data/raw/german


In [4]:
#Loading the actual dataset to local location from module ftp
!wget ftp://ftp.ics.uci.edu/pub/machine-learning-databases/statlog/german/german.data
!wget ftp://ftp.ics.uci.edu/pub/machine-learning-databases/statlog/german/german.doc

--2019-08-20 12:15:09--  ftp://ftp.ics.uci.edu/pub/machine-learning-databases/statlog/german/german.data
           => ‘german.data.2’
Resolving ftp.ics.uci.edu (ftp.ics.uci.edu)... 128.195.1.14
Connecting to ftp.ics.uci.edu (ftp.ics.uci.edu)|128.195.1.14|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/machine-learning-databases/statlog/german ... done.
==> SIZE german.data ... 79793
==> PASV ... done.    ==> RETR german.data ... done.
Length: 79793 (78K) (unauthoritative)

100%[======================================>] 79,793      --.-K/s   in 0.07s   

2019-08-20 12:15:10 (1.06 MB/s) - ‘german.data.2’ saved [79793]

--2019-08-20 12:15:11--  ftp://ftp.ics.uci.edu/pub/machine-learning-databases/statlog/german/german.doc
           => ‘german.doc.2’
Resolving ftp.ics.uci.edu (ftp.ics.uci.edu)... 128.195.1.14
Connecting to ftp.ics.uci.edu (ftp.ics.uci.edu)|128.195.1.14|:21... connected.
Logging in as a

In [5]:
%cd -
dataset_orig = GermanDataset(protected_attribute_names=['age'],
# this dataset also contains protected
# attribute for "sex" which we do not
# consider in this evaluation
privileged_classes=[lambda x: x >= 25],      # age >=25 is considered privileged
features_to_drop=['personal_status', 'sex']) # ignore sex-related attributes

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

/home/dsxuser/work


In [6]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.169905


In [7]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

In [8]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000


<font color='green'>Summary</font>

# <font color='red'>Question 2</font>
# Keras exercise
- Creating a Keras model by loading an inbuilt data set
- preprocessing input data
- building a Sequential Keras model and compiling the model with a training configuration
- model evaluation

In [36]:
try:
    __import__('keras')
except ImportError:
    pip.main(['install', 'keras']) 
    

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

# Data
- Reuters newswire dataset.
         *This dataset consists of 11,228 newswires from the Reuters news agency. Each wire is encoded as a sequence of word indexes. Moreover, each wire is categorised into one of 46 topics, which will serve as our label. This dataset is available through the Keras API.

In [37]:
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

seed = 1337
np.random.seed(seed)

In [38]:
from keras.datasets import reuters

max_words = 1000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2,
                                                         seed=seed)
num_classes = np.max(y_train) + 1 

In [39]:
# restore np.load for future normal usage
np.load = np_load_old

In [40]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

In [41]:
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

In [42]:
model = Sequential()  # Instantiate sequential model
model.add(Dense(512, activation='relu', input_shape = (max_words,))) # Add first layer. Make sure to specify input shape
model.add(Dropout(0.5)) # Add second layer
model.add(Dense(num_classes, activation='softmax')) # Add third layer

In [43]:
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

In [44]:
from keras import backend as K

K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))

In [45]:
batch_size = 32
model.fit(x_train, y_train, batch_size=batch_size, epochs=5, validation_data=(x_test,y_test))
score = model.evaluate(x_test,y_test, verbose=1)

Train on 8982 samples, validate on 2246 samples
Epoch 1/5
8982/8982 [==============================] - 9s 1ms/step - loss: 1.3969 - acc: 0.6885 - val_loss: 0.9784 - val_acc: 0.7765
Epoch 2/5
8982/8982 [==============================] - 9s 975us/step - loss: 0.7714 - acc: 0.8192 - val_loss: 0.8214 - val_acc: 0.8090
Epoch 3/5
8982/8982 [==============================] - 9s 947us/step - loss: 0.5568 - acc: 0.8652 - val_loss: 0.8174 - val_acc: 0.8099
Epoch 4/5
8982/8982 [==============================] - 9s 957us/step - loss: 0.4190 - acc: 0.8926 - val_loss: 0.8370 - val_acc: 0.8010
Epoch 5/5
2246/2246 [==============================] - 1s 275us/step


In [46]:
print("Loss: ",score[0],"\nAccuracy:",score[1])

Loss:  0.8556284526046216 
Accuracy: 0.798753339269813
